In [5]:
import pandas as pd
from sklearn import datasets, linear_model, metrics

In [8]:
train = pd.read_csv('../data/processed/train_data.csv')
train = pd.read_csv('../data/processed/test_data.csv')

In [ ]:
#split input and output

In [ ]:
#Linear regression: 
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)


In [ ]:
#predict sales price on test set
predictions = reg.predict(X_test)

In [ ]:
#Apply inverse transformation to bring predictions back to the original scale
predictions = np.exp(predictions)
predictions[:5]  